In [27]:
!pip3 install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [28]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import slugify
import os

clubs_url = "http://www.cpliege.be/caleclub.asp"

clubs_url_html = requests.get(clubs_url).text

soup = BeautifulSoup(clubs_url_html, "html.parser")

# read html and get every links
clubs = soup.find_all("a")

# tranform to get a dict with club name and url
clubs_dict = {}
for club in clubs:
    # removes "all whitespace characters (space, tab, newline, return, formfeed)"
    club_name = " ".join(club.text.split())
    clubs_dict[club_name] = "http://www.cpliege.be/" + club["href"]

print(clubs_dict)


{'44 RBC ESNEUX': 'http://www.cpliege.be/clubs/club0044.asp', '493 RABC ENSIVAL': 'http://www.cpliege.be/clubs/club0493.asp', '1338 REN. MONTEGNEE': 'http://www.cpliege.be/Clubs/club1338.asp', '2410 ETOILE JUPILLE': 'http://www.cpliege.be/Clubs/club2410.asp', '2732 BC OLNE': 'http://www.cpliege.be/clubs/club2732.asp', '46 PEPINSTER EAST B': 'http://www.cpliege.be/clubs/club0046.asp', '524 R 4A AYWAILLE-HAMOIR': 'http://www.cpliege.be/clubs/club0524.asp', '1377 HENRI CHAPELLE': 'http://www.cpliege.be/Clubs/club1377.asp', '2440 BCM HERSTAL': 'http://www.cpliege.be/Clubs/club2440.asp', '2733 ESL UNITED': 'http://www.cpliege.be/clubs/club2733.asp', '57 ATLAS A JUPILLE': 'http://www.cpliege.be/clubs/club0057.asp', '550 ROYAL BC WANZE': 'http://www.cpliege.be/clubs/club0550.asp', '1388 RBC AUBEL': 'http://www.cpliege.be/Clubs/club1388.asp', '2458 BBC EUPEN': 'http://www.cpliege.be/Clubs/club2458.asp', '2744 MARCHIN BASKET': 'http://www.cpliege.be/Clubs/club2744.asp', '68 RB UNION LIEGE': 'ht

In [29]:
def get_club_agenda(club_url):
    agenda = pd.read_html(club_url, header=5, encoding="ISO-8859-1")[0]

    # remove if column "Unnamed: 7" is empty OR starts with "(" and ends with ")"
    agenda = agenda[
        ~(
            agenda["Unnamed: 7"].isnull()
            | agenda["Unnamed: 7"].str.startswith("(")
            & agenda["Unnamed: 7"].str.endswith(")")
        )
    ]

    print(f"Agenda shape: {agenda.shape} from {club_url}")

    # rename columns
    agenda.columns = [
        "Code",
        "Unknown",
        "Weekday",
        "Date",
        "Heure",
        "Équipe 1",
        "Équipe 2",
        "Catégorie",
        "Autre",
    ]

    # drop "Unknown" column
    agenda.drop("Unknown", axis=1, inplace=True)

    agenda = agenda[~agenda["Date"].isnull()]
    agenda = agenda[~agenda["Heure"].isnull()]

    # Heure to string
    agenda["Heure"] = agenda["Heure"].astype(str)

    # if Heure is ".", set it to 00:00
    agenda.loc[agenda["Heure"] == ".", "Heure"] = "00:00"

    # replace . and ; in Heure by :
    for char in [".", ";"]:
        agenda["Heure"] = agenda["Heure"].str.replace(char, ":", regex=False)

    # if Heure contains only one number after : add a 0
    agenda["Heure"] = agenda["Heure"].apply(
        lambda x: x if len(x.split(":")[1]) == 2 else x + "0"
    )

    # remove Weekday column
    agenda.drop("Weekday", axis=1, inplace=True)

    # Date as datetime
    agenda["Date"] = pd.to_datetime(agenda["Date"], format="%d/%m/%y")

    # order by catégorie and then by date
    agenda.sort_values(by=["Catégorie", "Date"], inplace=True)

    return agenda

In [30]:
from icalendar import Calendar, Event
import uuid
from datetime import datetime


def generate_ics(agenda, filename, cal_name, club_url):
    cal = Calendar()

    cal.add("version", "2.0")

    for event in agenda.iterrows():
        e = Event()

        name = (
            "🏀 "
            + event[1]["Catégorie"]
            + ": "
            + event[1]["Équipe 1"]
            + " et "
            + event[1]["Équipe 2"]
        )

        code = event[1]["Code"]
        description = "[" + code + "] — " + name

        startime = (
            pd.to_datetime(event[1]["Date"]).strftime("%Y-%m-%d")
            + " "
            + event[1]["Heure"]
        )
        endtime = pd.to_datetime(startime) + pd.Timedelta(minutes=120)

        # time zone
        location = "Europe/Brussels"

        # from local time to UTC
        startime = pd.to_datetime(startime).tz_localize(location).tz_convert("UTC")
        endtime = pd.to_datetime(endtime).tz_localize(location).tz_convert("UTC")

        # if ["Équipe 1"] is not empty
        if not pd.isnull(event[1]["Équipe 1"]):
            location = event[1]["Équipe 1"]
        else:
            location = ""

        # generate uuid from name
        event_uuid = uuid.uuid5(uuid.NAMESPACE_DNS, name)

        e.begin = startime.strftime("%Y-%m-%d %H:%M:%S%Z")
        e.end = endtime.strftime("%Y-%m-%d %H:%M:%S%Z")
        e.add("summary", name)
        e.add("dtstart", startime)
        e.add("dtend", endtime)
        # e.add('dtstamp', datetime.now())
        e.add("location", location)
        e.add("priority", 5)
        e.add("sequence", 1)
        e.add("description", description)
        e.add("url", club_url)
        e.add("uid", event_uuid)

        cal.add_component(e)

    # save to file
    with open(filename, "wb") as f:
        f.write(cal.to_ical())
        f.close()

In [31]:
base_raw_path = (
    "https://raw.githubusercontent.com/tintamarre/sport-events-to-calendar/main/"
)

md = "# 🏀 Les clubs du CPLiège\n\n"

md += "Ce dépôt contient les agendas des clubs de basket du CPLiège.\n\n"
md += "Les agendas sont disponibles au format CSV et ICS.\n\n"
md += "Les agendas sont mis à jour automatiquement toutes semaines.\n\n"
md += "[L'agenda global](https://raw.githubusercontent.com/tintamarre/sport-events-to-calendar/main/data/CPLi%C3%A8ge.ics) est également disponible.\n\n"
md += "Dernière mise à jour: " + datetime.now().strftime("%d/%m/%Y %H:%M:%S") + "\n\n"

agenda = pd.DataFrame()

for club_name, club_url in clubs_dict.items():
    md += "## [" + club_name + "](" + club_url + ")\n\n"

    club_agenda = get_club_agenda(club_url)

    agenda = pd.concat([agenda, club_agenda], ignore_index=True)

    storage_path = "data/" + slugify.slugify(club_name)

    if not os.path.exists(storage_path):
        os.mkdir(storage_path)

    club_agenda.to_csv(
        storage_path + "/" + slugify.slugify(club_name) + ".csv", index=False
    )

    md += (
        "* [Agenda]("
        + base_raw_path
        + storage_path
        + "/"
        + slugify.slugify(club_name)
        + ".csv)\n"
    )

    categories = club_agenda["Catégorie"].unique()

    for category in categories:
        club_category_agenda = club_agenda[club_agenda["Catégorie"] == category]

        filename = (
            storage_path
            + "/"
            + slugify.slugify(club_category_agenda["Catégorie"].iloc[0])
            + ".ics"
        )

        cal_name = club_name + " - " + club_category_agenda["Catégorie"].iloc[0]

        print(f"Generating {cal_name} calendar at {filename}")
        generate_ics(club_category_agenda, filename, cal_name, club_url)

        md += (
            "* ["
            + category
            + "]("
            + base_raw_path
            + storage_path
            + "/"
            + slugify.slugify(category)
            + ".ics)\n"
        )

    md += "\n"

# remove duplicates from agenda
agenda.drop_duplicates(inplace=True)
generate_ics(
    agenda, "data/CPLiège.ics", "CPLiège", "http://www.cpliege.be/caleclub.asp"
)

with open("listing.md", "w") as f:
    f.write(md)
    f.close()

/Users/martin/Library/Python/3.9/lib/python/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


Agenda shape: (294, 9) from http://www.cpliege.be/clubs/club0044.asp
Generating 44 RBC ESNEUX - AW226001 calendar at data/44-rbc-esneux/aw226001.ics
Generating 44 RBC ESNEUX - AW226005 calendar at data/44-rbc-esneux/aw226005.ics
Generating 44 RBC ESNEUX - AWBB DD - POULE F calendar at data/44-rbc-esneux/awbb-dd-poule-f.ics
Generating 44 RBC ESNEUX - CPE PROV DAMES calendar at data/44-rbc-esneux/cpe-prov-dames.ics
Generating 44 RBC ESNEUX - DD - DIVISION 1 calendar at data/44-rbc-esneux/dd-division-1.ics
Generating 44 RBC ESNEUX - DD - DIVISION 3 A calendar at data/44-rbc-esneux/dd-division-3-a.ics
Generating 44 RBC ESNEUX - DD - REGIONALE 2 A calendar at data/44-rbc-esneux/dd-regionale-2-a.ics
Generating 44 RBC ESNEUX - R 4A AYWAILLE-HAMOIR A (+ 5) calendar at data/44-rbc-esneux/r-4a-aywaille-hamoir-a-5.ics
Generating 44 RBC ESNEUX - U 10 - MM C calendar at data/44-rbc-esneux/u-10-mm-c.ics
Generating 44 RBC ESNEUX - U 10 - MM D calendar at data/44-rbc-esneux/u-10-mm-d.ics
Generating 44

/var/folders/zn/0wszm2k93n1_tvmfqd8t1_sc0000gn/T/ipykernel_3934/4233784187.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agenda.drop("Unknown", axis=1, inplace=True)


Generating 44 RBC ESNEUX - U 19 - DD A calendar at data/44-rbc-esneux/u-19-dd-a.ics
Generating 44 RBC ESNEUX - U 7 - MM A calendar at data/44-rbc-esneux/u-7-mm-a.ics
Generating 44 RBC ESNEUX - U 8 - MM E calendar at data/44-rbc-esneux/u-8-mm-e.ics


/Users/martin/Library/Python/3.9/lib/python/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


...

Generating 1298 BC COINTE - U 7 - MM B calendar at data/1298-bc-cointe/u-7-mm-b.ics
Generating 1298 BC COINTE - U 8 - MM C calendar at data/1298-bc-cointe/u-8-mm-c.ics


/Users/martin/Library/Python/3.9/lib/python/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
/var/folders/zn/0wszm2k93n1_tvmfqd8t1_sc0000gn/T/ipykernel_3934/4233784187.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agenda.drop("Unknown", axis=1, inplace=True)


Agenda shape: (27, 9) from http://www.cpliege.be/Clubs/club2329.asp
Generating 2329 BC TORES LIEGE - CPE PROV MESSIEURS calendar at data/2329-bc-tores-liege/cpe-prov-messieurs.ics
Generating 2329 BC TORES LIEGE - MM - DIVISION 4 D calendar at data/2329-bc-tores-liege/mm-division-4-d.ics


/Users/martin/Library/Python/3.9/lib/python/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
/var/folders/zn/0wszm2k93n1_tvmfqd8t1_sc0000gn/T/ipykernel_3934/4233784187.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agenda.drop("Unknown", axis=1, inplace=True)


Agenda shape: (54, 9) from http://www.cpliege.be/clubs/club2731.asp
Generating 2731 BC REVEIL - CPE PROV MESSIEURS calendar at data/2731-bc-reveil/cpe-prov-messieurs.ics
Generating 2731 BC REVEIL - MM - DIVISION 3 C calendar at data/2731-bc-reveil/mm-division-3-c.ics
Generating 2731 BC REVEIL - MM - DIVISION 4 E calendar at data/2731-bc-reveil/mm-division-4-e.ics
